# How much gets filtered when removing rRNA, tRNA, and mtDNA/RNA? 
# Notebook 1: fetching the data
I'm making this notebook as a n intermission from preparing rolypoly external database. So far I've used a combination of NCBI +SILVA rRNAs to remove contaminating rRNA reads from RNA-seq data. Subsequently, the organisms whose rRNAs were most matched, are then also fetched (or their transcriptomes, if available) to remove any remaining reads that may have come from those organisms.  
THis is messy, requires NCBI taxdump, taxonkit, and ncbi-datasets. So I started removing this dependency by using a set of rRNAs for which I can generate a prebuilt table containing the FTP addresses of the hosts' genomes/transcriptomes.  
While doing so, I realised the step above could be split - quick rRNA mapping to get rough taxonomic breakdown of the sample, and then a more thorough removal of rRNA, tRNA, and mtDNA/RNA using a more comprehensive database. The question is then how much will these diffrent combinations filter, how much more time, and would masking the fasta for subsequences shared with RNA viruses change the results significantly?

To test these, below are how I got the data, how I named the sets. For simplicity, the [trrna_fetch.ipynb](trrna_fetch.ipynb) has the code for the actual filtering using these sequence typs, graphs and so on. 

*Note*: Parts of this script are from [build_data.py](../src/rolypoly/commands/misc/build_data.py) and [filter_reads.py](../src/rolypoly/commands/reads/filter_reads.py) scripts in the rolypoly repository.

Loading libraries and defining paths to sets already created/downloaded:

In [ ]:
%load_ext autoreload
%autoreload 2
import hashlib
import os
from ftplib import FTP
import glob

import polars as pl
from bbmapy import bbduk

from rolypoly.utils.bio.sequences import write_fasta_file, remove_duplicates


from rolypoly.utils.bio.polars_fastx import from_fastx_eager

from rolypoly.utils.logging.loggit import setup_logging
from rolypoly.utils.various import fetch_and_extract, run_command_comp, extract

### DEBUG ARGS (for manually building, not entering via CLI):
threads = 6
log_file = "notebooks/Exprimental/trrna.log"
global data_dir
data_dir = "/clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data"
os.environ["ROLYPOLY_DATA_DIR"] = data_dir

global rrna_dir
global contam_dir

logger = setup_logging(log_file)
print(f"Starting data preparation to : {data_dir}")

contam_dir = os.path.join(data_dir, "contam")
os.makedirs(contam_dir, exist_ok=True)

rrna_dir = os.path.join(contam_dir, "rrna")
os.makedirs(rrna_dir, exist_ok=True)

trna_dir = os.path.join(contam_dir, "trna")
os.makedirs(trna_dir, exist_ok=True)

masking_dir = os.path.join(contam_dir, "masking")
os.makedirs(masking_dir, exist_ok=True)

reference_seqs = os.path.join(data_dir, "reference_seqs")
os.makedirs(reference_seqs, exist_ok=True)

mmseqs_ref_dir = os.path.join(reference_seqs, "mmseqs")
os.makedirs(mmseqs_ref_dir, exist_ok=True)

rvmt_dir = os.path.join(reference_seqs, "RVMT")
os.makedirs(rvmt_dir, exist_ok=True)

ncbi_ribovirus_dir = os.path.join(reference_seqs, "ncbi_ribovirus")
os.makedirs(ncbi_ribovirus_dir, exist_ok=True)

# Masking sequences preparation
rvmt_fasta_path = os.path.join(
    data_dir, "reference_seqs", "RVMT", "RVMT_cleaned_contigs.fasta"
)
ncbi_ribovirus_fasta_path = os.path.join(
    data_dir,
    "reference_seqs",
    "ncbi_ribovirus",
    "refseq_ribovirus_genomes.fasta",
)

rna_viruses_entropy_masked_path = os.path.join(
    masking_dir, "combined_entropy_masked.fasta"
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Starting data preparation to : /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data


## Fetching data (SILVA, NCBI rRNAs, tRNAs, mtDNAs):

In [29]:
silva_release = "138.2"

# Download SILVA rRNA sequences (SSU and LSU)
silva_ssu_path = os.path.join(
    rrna_dir, f"SILVA_{silva_release}_SSURef_NR99_tax_silva.fasta"
)
silva_lsu_path = os.path.join(
    rrna_dir, f"SILVA_{silva_release}_LSURef_NR99_tax_silva.fasta"
)
# skipping downloading (already done)
fetch_and_extract(
    f"https://www.arb-silva.de/fileadmin/silva_databases/release_{silva_release.replace('.', '_')}/Exports/SILVA_{silva_release}_SSURef_NR99_tax_silva.fasta.gz",
    fetched_to=os.path.join(rrna_dir, "tmp_ssu.fasta.gz"),
    extract_to=rrna_dir,
    rename_extracted=silva_ssu_path,
    logger=logger,
)
fetch_and_extract(
    f"https://www.arb-silva.de/fileadmin/silva_databases/release_{silva_release.replace('.', '_')}/Exports/SILVA_{silva_release}_LSURef_NR99_tax_silva.fasta.gz",
    fetched_to=os.path.join(rrna_dir, "tmp_lsu.fasta.gz"),
    extract_to=rrna_dir,
    rename_extracted=silva_lsu_path,
    logger=logger,
)

# Download SILVA taxonomy mappings (maps accessions to NCBI taxids)
silva_ssu_taxmap = pl.read_csv(
    "https://www.arb-silva.de/fileadmin/silva_databases/current/Exports/taxonomy/ncbi/taxmap_embl-ebi_ena_ssu_ref_nr99_138.2.txt.gz",
    truncate_ragged_lines=True,
    separator="\t",
    infer_schema_length=123123,
)
silva_lsu_taxmap = pl.read_csv(
    "https://www.arb-silva.de/fileadmin/silva_databases/current/Exports/taxonomy/ncbi/taxmap_embl-ebi_ena_lsu_ref_nr99_138.2.txt.gz",
    truncate_ragged_lines=True,
    separator="\t",
    infer_schema_length=123123,
)
silva_taxmap = pl.concat([silva_lsu_taxmap, silva_ssu_taxmap])
silva_taxmap.write_parquet(
    os.path.join(rrna_dir, "silva_taxmap_embl-ebi_ena.parquet")
)

INFO     2025-12-28 22:53:33 - Downloading                                                                                              ]8;id=664730;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py\various.py]8;;\:]8;id=737937;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py#215\215]8;;\
         https://www.arb-silva.de/fileadmin/silva_databases/release_138_2/Exports/SILVA_138.2_SSURef_NR99_tax_silva.fasta.gz to                       
         /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/tmp_ssu.fasta.gz                                                  

INFO     2025-12-28 22:54:01 - Successfully downloaded to                                                                               ]8;id=74975;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py\various.py]8;;\:]8;id=787093;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py#221\221]8;;\
         /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/tmp_ssu.fasta.gz                                                  

INFO     2025-12-28 22:54:01 - Extracting /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/tmp_ssu.fasta.gz   ]8;id=479798;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py\various.py]8;;\:]8;id=958292;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py#255\255]8;;\
         to /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna                                                                

INFO     2025-12-28 22:54:05 - Successfully extracted to                                                                                ]8;id=463229;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py\various.py]8;;\:]8;id=894040;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py#288\288]8;;\
         /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/tmp_ssu.fasta                                                     

INFO     2025-12-28 22:54:05 - Downloading                                                                                              ]8;id=306869;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py\various.py]8;;\:]8;id=962956;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py#215\215]8;;\
         https://www.arb-silva.de/fileadmin/silva_databases/release_138_2/Exports/SILVA_138.2_LSURef_NR99_tax_silva.fasta.gz to                       
         /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/tmp_lsu.fasta.gz                                                  

INFO     2025-12-28 22:54:13 - Successfully downloaded to                                                                               ]8;id=789217;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py\various.py]8;;\:]8;id=399592;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py#221\221]8;;\
         /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/tmp_lsu.fasta.gz                                                  

INFO     2025-12-28 22:54:13 - Extracting /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/tmp_lsu.fasta.gz   ]8;id=673944;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py\various.py]8;;\:]8;id=359225;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py#255\255]8;;\
         to /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna                                                                

INFO     2025-12-28 22:54:15 - Successfully extracted to                                                                                ]8;id=769819;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py\various.py]8;;\:]8;id=785533;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py#288\288]8;;\
         /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/tmp_lsu.fasta                                                     

In [30]:
# Parse SILVA headers and extract accessions
silva_fasta_df = pl.concat(
    [
        from_fastx_eager(silva_ssu_path).with_columns(
            pl.lit("SSU").alias("rRNA_type")
        ),
        from_fastx_eager(silva_lsu_path).with_columns(
            pl.lit("LSU").alias("rRNA_type")
        ),
    ]
)
silva_fasta_df.write_parquet(os.path.join(rrna_dir, "silva99_fasta.parquet"))
print(f"total SILVA99 sequences {silva_fasta_df.height}")
silva_fasta_df.head(4)

total SILVA99 sequences 605774


header,sequence,rRNA_type
str,str,str
"""AY846379.1.1791 Eukaryota;Arch…","""AACCUGGUUGAUCCUGCCAGUAGUCAUAUG…","""SSU"""
"""AY846382.1.1778 Eukaryota;Arch…","""GUUGAUCCUGCCAGUAGUCAUAUGCUUGUC…","""SSU"""
"""AB000393.1.1510 Bacteria;Pseud…","""UGGCUCAGAUUGAACGCUGGCGGCAGGCCU…","""SSU"""
"""AY909590.1.2352 Eukaryota;Arch…","""GACUAAGCCAUGCAUGUCUAAGUAUAAACG…","""SSU"""


In [31]:
# Extract accession from header (format: >accession.version rest_of_header)
silva_fasta_df = silva_fasta_df.with_columns(
    primaryAccession=pl.col("header").str.extract(
        r"^([A-Za-z0-9_]+)(?:\.\d+)*", 1
    ),  # DQ150555.1.2478 -> DQ150555
    accession=pl.col("header").str.extract(
        r"^([A-Za-z0-9_]+(?:\.\d+)?)", 1
    ),  # AY846379 or DQ150555.1
    taxonomy_raw=pl.col("header").str.replace(r"^\S+\s+", ""),
)
# silva_fasta_df = silva_fasta_df.with_columns(
#     pl.col("sequence").str.len_chars().alias("seq_length")
# )
# silva_taxmap = silva_taxmap.with_columns(
#     (pl.col("stop") - pl.col("start")).alias("seq_length")
# )

silva_df = silva_fasta_df.join(
    silva_taxmap.select(
        ["primaryAccession", "ncbi_taxonid", "submitted_path"]
    ).unique(),  # seq_length
    on=["primaryAccession"],
    how="inner",
)
# silva_df.height
silva_df["ncbi_taxonid"].null_count()

# Load SILVA taxonomy mappings
print(
    f"Merged taxonomy for {silva_df.filter(pl.col('ncbi_taxonid').is_not_null()).height} SILVA sequences"
)

unique_taxids = (
    silva_df.filter(pl.col("ncbi_taxonid").is_not_null())
    .select("ncbi_taxonid")
    .unique()["ncbi_taxonid"]
    .to_list()
)
print(
    f"Total of {len(unique_taxids)} unique NCBI taxids found in SILVA sequences"
)

Merged taxonomy for 605774 SILVA sequences
Total of 105645 unique NCBI taxids found in SILVA sequences


In [32]:
silva_df["ncbi_taxonid"].value_counts(sort=True).head(10)

ncbi_taxonid,count
i64,u32
77133,183357
155900,19293
115547,12243
256318,7959
562,7811
100272,6908
136703,5459
573,4190
1280,3213


Removing 77133,155900,115547,256318,100272,136703,171953,86473 (uncultured) as they are not useful for genome fetching later on.
Also removing things wiht "uncultured" in the submitted_path.

In [33]:
silva_df = silva_df.with_columns(
    t1=pl.concat_str(
        [pl.col("ncbi_taxonid"), pl.lit("silva"), pl.col("rRNA_type")],
        separator="@",
    )
).with_columns(
    new_header=pl.concat_str([pl.col("t1"), pl.col("header")], separator=" ")
)
silva_df = silva_df.with_columns(
    t1=pl.concat_str(
        [pl.col("ncbi_taxonid"), pl.lit("silva"), pl.col("rRNA_type")],
        separator="@",
    )
)

silva_df = silva_df.with_columns(
    pl.col("sequence")
    .map_elements(lambda x: hashlib.md5(x.encode()).hexdigest())
    .alias("seq_hash")
).with_columns(
    new_header=pl.concat_str([pl.col("t1"), pl.col("seq_hash")], separator="@")
)
silva_df = silva_df.unique()
silva_df = silva_df.unique(subset=["sequence"])
silva_df = silva_df.drop("t1")

silva_df["new_header"].head(1).item()

heigt_befrore = silva_df.height
print(silva_df.height)
silva_df = silva_df.filter(
    ~pl.col("ncbi_taxonid").is_in(
        [77133, 155900, 115547, 256318, 100272, 136703, 171953, 86473, 56765]
    )
)
silva_df = silva_df.filter(
    ~pl.col("submitted_path").str.contains_any(
        ["uncultured", "unclassified", "synthetic", "environmental sample"]
    )
)

print(silva_df.height)
for coln in silva_df.columns:
    print(coln)
    print(silva_df[coln].sample(5).to_list())

silva_df.write_parquet(os.path.join(rrna_dir, "silva_rrna_sequences.parquet"))
print("Finished writing SILVA rRNA sequences parquet file.")
print(
    f"total SILVA sequences after filtering: {silva_df.height}, removed {heigt_befrore - silva_df.height} sequences"
)

530930
226832
header
['FN773278.1.1468 Bacteria;Campylobacterota;Campylobacteria;Campylobacterales;Sulfurospirillaceae;Sulfurospirillum;bacterium endosymbiont of Osedax mucofloris', 'AF163120.1.1458 Bacteria;Actinomycetota;Actinobacteria;Streptosporangiales;Thermomonosporaceae;Actinoallomurus;Actinoallomurus spadix', 'CP022663.4249352.4250905 Bacteria;Pseudomonadota;Gammaproteobacteria;Enterobacterales;Enterobacteriaceae;Salmonella;Salmonella enterica', 'EU136629.1.1439 Bacteria;Pseudomonadota;Gammaproteobacteria;Enterobacterales;Pseudoalteromonadaceae;Pseudoalteromonas;Pseudoalteromonas sp. Z6', 'KC984863.1.1291 Bacteria;Pseudomonadota;Gammaproteobacteria;Pseudomonadales;Halomonadaceae;Halomonas;Halomonas sp. UADY-002']
sequence
['AACCUGGUUGAUCCUGCCAGUAGUCACUCGCUCGUCUCAAAGACUAAGCCAUGCAUGUCUAAGUAUAAAUCUUUUACUUUGAAACUGCGAACGGCUCAUUAUAUCAGUUAUAGUUUAUUUGAUAGUCCCUUACUACUUGGAUAACCGUAGUAAUUCUAGAGCUAAUACAUGCGUCAAUACCCUUCUGGGGUAGUAUUUAUUAGAUUGAAACCAACCCCUUCGGGGUGAUGUGGUGAUUCAUAAUAAGCUUGCGGAUCG

Deduplicate, mask kmers shared with RNA viruses, and apply entrophy masking.

In [34]:
import rich_click as click

# replace headers with md5 hash as the new_header breaks mappers. Will remap after this
silva_df = silva_df.with_columns(
    pl.col("sequence")
    .map_elements(lambda x: hashlib.md5(x.encode()).hexdigest())
    .alias("tmp_header")
)

write_fasta_file(
    headers=silva_df["tmp_header"].to_list(),
    seqs=silva_df["sequence"].to_list(),
    output_file=os.path.join(rrna_dir, "temp_silva_rrna_sequences.fasta"),
)
# deduplicate sequences
deduplicated_silva_fasta_path = os.path.join(
    rrna_dir, "silva_rrna_sequences_deduplicated.fasta"
)
remove_duplicates(
    input_file=os.path.join(rrna_dir, "temp_silva_rrna_sequences.fasta"),
    output_file=deduplicated_silva_fasta_path,
    logger=logger,
    by="seq",
    # revcomp_as_distinct=False
)

INFO     2025-12-28 22:55:14 - Processed 226832 records: 226832 unique, 0 duplicates removed                                          ]8;id=852034;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/bio/sequences.py\sequences.py]8;;\:]8;id=718033;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/bio/sequences.py#725\725]8;;\

In [35]:
# mask shared kmers with RNA viruses
masked_silva_fasta_path = os.path.join(
    rrna_dir, "silva_rrna_sequences_deduplicated_masked.fasta"
)
entrophy_masked_silva_fasta_path = os.path.join(
    rrna_dir, "silva_rrna_sequences_deduplicated_masked_entropy.fasta"
)

mask_args = {
    "threads": 14,
    "memory": "18gb",
    "output": str(masked_silva_fasta_path),
    "flatten": False,
    "aligner": "mmseqs2",
    "input": str(deduplicated_silva_fasta_path),
}

# ensure we have the click Command object (the package import may have bound the module)
from rolypoly.commands.reads.mask_dna import mask_dna as _mask_cmd

# If it's a click.Command use a Context to invoke, otherwise call directly
if isinstance(_mask_cmd, click.core.Command):
    ctx = click.Context(_mask_cmd)
    ctx.invoke(_mask_cmd, **mask_args)
else:
    # fallback: call the function directly
    _mask_cmd(**mask_args)

# # Apply entropy masking
bbduk(
    in1=masked_silva_fasta_path,
    out=entrophy_masked_silva_fasta_path,
    entropy=0.4,
    entropyk=4,
    entropywindow=24,
    maskentropy=True,
    ziplevel=9,
)


# read the masked fasta and remap to original headers
silva_rrna_masked_df = from_fastx_eager(
    entrophy_masked_silva_fasta_path
).rename({"header": "tmp_header", "sequence": "masked_sequence"})
silva_rrna_masked_df = silva_rrna_masked_df.join(
    silva_df, on=["tmp_header"], how="inner"
)
silva_rrna_masked_df.write_parquet(
    os.path.join(rrna_dir, "silva_rrna_sequences_masked.parquet")
)
print("Finished writing SILVA rRNA sequences parquet file.")
write_fasta_file(
    headers=silva_rrna_masked_df["new_header"].to_list(),
    seqs=silva_rrna_masked_df["masked_sequence"].to_list(),
    output_file=os.path.join(
        rrna_dir, "silva_rRNA_all_sequences_masked_entropy.fasta"
    ),
)
print(
    f"finished writing masked SILVA rRNA fasta file: {os.path.join(rrna_dir, 'silva_rRNA_all_sequences_masked_entropy.fasta')}"
)

INFO     2025-12-28 22:55:34 - datadir used: /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data                             ]8;id=325631;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/commands/reads/mask_dna.py\mask_dna.py]8;;\:]8;id=88559;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/commands/reads/mask_dna.py#71\71]8;;\

Running command: mmseqs easy-linsearch /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/masking/combined_entropy_masked.fasta /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/silva_rrna_sequences_deduplicated.fasta /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/tmp_mask_dna/tmp_mapped.sam /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/tmp_mask_dna  --min-seq-id 0.7 --min-aln-len 80 --threads 14 --format-mode 1 --search-type 3 -v 3 --max-accept 1231 -a  
easy-linsearch /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/masking/combined_entropy_masked.fasta /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/silva_rrna_sequences_deduplicated.fasta /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/tmp_mask_dna/tmp_mapped.sam /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/tmp_mask_dna --min-seq-

File '/clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/tmp_mask_dna/tmp_mapped.sam' size 
is 236303 bytes (not empty). 

INFO     2025-12-28 22:56:03 - Finished running aligner mmseqs2                                                                        ]8;id=816392;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/commands/reads/mask_dna.py\mask_dna.py]8;;\:]8;id=867661;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/commands/reads/mask_dna.py#220\220]8;;\

INFO     2025-12-28 22:56:03 - beginning bbmask (masking + entropy) step                                                               ]8;id=611612;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/commands/reads/mask_dna.py\mask_dna.py]8;;\:]8;id=125819;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/commands/reads/mask_dna.py#221\221]8;;\

...

Result:                         226832 reads (100.00%)  431916037 bases (100.00%)

Time:                           1.808 seconds.

Reads Processed:        226k    125.46k reads/sec

Bases Processed:        431m    238.89m bases/sec

Finished writing SILVA rRNA sequences parquet file.
finished writing masked SILVA rRNA fasta file: /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/silva_rRNA_all_sequences_masked_entropy.fasta


In [ ]:
fetch_and_extract(
    url="https://ftp.ncbi.nlm.nih.gov/genomes/genbank/assembly_summary_genbank.txt",
    fetched_to=os.path.join(rrna_dir, "assembly_summary_genbank.txt.gz"),
    extract=False,
)
print("Loading NCBI GenBank assembly summary")
# genbank_summary = pl.read_csv(os.path.join(rrna_dir, "assembly_summary_genbank.txt.gz",),
# infer_schema_length=100020, separator="\t", skip_rows=1,
# null_values=["na","NA","-"],ignore_errors=True,
# has_header=True)
# polars failed me, so using line by line iterator
from gzip import open as gz_open

with gz_open(
    os.path.join(rrna_dir, "assembly_summary_genbank.txt.gz"), "r"
) as f:
    header = None
    records = []
    i = 0
    for line in f:
        if i == 0:
            i += 1
            continue
        line = line.rstrip(b"\n")
        if i == 1:
            header = line.decode()[1:].strip().split("\t")
            i += 1
            continue
        fields = line.decode().strip().split("\t")
        record = dict(zip(header, fields))
        records.append(record)
genbank_summary = pl.from_records(records).rename({"taxid": "ncbi_taxonid"})
genbank_summary.collect_schema()
# Schema([('assembly_accession', String),
#         ('bioproject', String),
#         ('biosample', String),
#         ('wgs_master', String),
#         ('refseq_category', String),
#         ('ncbi_taxonid', String),
#         ('species_taxid', String),
#         ('organism_name', String),
#         ('infraspecific_name', String),
#         ('isolate', String),
#         ('version_status', String),
#         ('assembly_level', String),
#         ('release_type', String),
#         ('genome_rep', String),
#         ('seq_rel_date', String),
#         ('asm_name', String),
#         ('asm_submitter', String),
#         ('gbrs_paired_asm', String),
#         ('paired_asm_comp', String),
#         ('ftp_path', String),
#         ('excluded_from_refseq', String),
#         ('relation_to_type_material', String),
#         ('asm_not_live_date', String),
#         ('assembly_type', String),
#         ('group', String),
#         ('genome_size', String),
#         ('genome_size_ungapped', String),
#         ('gc_percent', String),
#         ('replicon_count', String),
#         ('scaffold_count', String),
#         ('contig_count', String),
#         ('annotation_provider', String),
#         ('annotation_name', String),
#         ('annotation_date', String),
#         ('total_gene_count', String),
#         ('protein_coding_gene_count', String),
#         ('non_coding_gene_count', String),
#         ('pubmed_id', String)])

genbank_summary.write_csv(
    os.path.join(rrna_dir, "genbank_assembly_summary.tsv"), separator="\t"
)
genbank_summary = pl.read_csv(
    os.path.join(rrna_dir, "genbank_assembly_summary.tsv"),
    infer_schema_length=100020,
    separator="\t",
    null_values=["na", "NA", "-"],
    ignore_errors=True,
    has_header=True,
)

# In [91]: genbank_summary.collect_schema()
# Out[91]:
# Schema([('assembly_accession', String),
#         ('bioproject', String),
#         ('biosample', String),
#         ('wgs_master', String),
#         ('refseq_category', String),
#         ('ncbi_taxonid', Int64),
#         ('species_taxid', Int64),
#         ('organism_name', String),
#         ('infraspecific_name', String),
#         ('isolate', String),
#         ('version_status', String),
#         ('assembly_level', String),
#         ('release_type', String),
#         ('genome_rep', String),
#         ('seq_rel_date', String),
#         ('asm_name', String),
#         ('asm_submitter', String),
#         ('gbrs_paired_asm', String),
#         ('paired_asm_comp', String),
#         ('ftp_path', String),
#         ('excluded_from_refseq', String),
#         ('relation_to_type_material', String),
#         ('asm_not_live_date', String),
#         ('assembly_type', String),
#         ('group', String),
#         ('genome_size', Int64),
#         ('genome_size_ungapped', Int64),
#         ('gc_percent', Float64),
#         ('replicon_count', Int64),
#         ('scaffold_count', Int64),
#         ('contig_count', Int64),
#         ('annotation_provider', String),
#         ('annotation_name', String),
#         ('annotation_date', String),
#         ('total_gene_count', Int64),
#         ('protein_coding_gene_count', Int64),
#         ('non_coding_gene_count', Int64),
#         ('pubmed_id', String)])

genbank_summary.write_parquet(
    os.path.join(rrna_dir, "genbank_assembly_summary.parquet")
)

INFO     2025-12-28 21:12:54 - Downloading https://ftp.ncbi.nlm.nih.gov/genomes/genbank/assembly_summary_genbank.txt to                 ]8;id=200560;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py\various.py]8;;\:]8;id=689039;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py#215\215]8;;\
         /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/assembly_summary_genbank.txt.gz                                   

INFO     2025-12-28 21:13:31 - Successfully downloaded to                                                                               ]8;id=244576;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py\various.py]8;;\:]8;id=981640;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py#221\221]8;;\
         /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/assembly_summary_genbank.txt.gz                                   

Loading NCBI GenBank assembly summary


Now the same for refseq as I'm not sure genbank contain all of the stuff in refseq?


In [ ]:
fetch_and_extract(
    url="https://ftp.ncbi.nlm.nih.gov/genomes/refseq/assembly_summary_refseq.txt",
    fetched_to=os.path.join(rrna_dir, "assembly_summary_refseq.txt.gz"),
    extract=False,
)
print("Loading NCBI RefSeq assembly summary")
# refseq_summary = pl.read_csv(os.path.join(rrna_dir, "assembly_summary_refseq.txt.gz",),
# infer_schema_length=100020, separator="\t", skip_rows=1,
# null_values=["na","NA","-"],ignore_errors=True,
# has_header=True)
# polars failed me, so using line by line iterator
from gzip import open as gz_open

with gz_open(
    os.path.join(rrna_dir, "assembly_summary_refseq.txt.gz"), "r"
) as f:
    header = None
    records = []
    i = 0
    for line in f:
        if i == 0:
            i += 1
            continue
        line = line.rstrip(b"\n")
        if i == 1:
            header = line.decode()[1:].strip().split("\t")
            i += 1
            continue
        fields = line.decode().strip().split("\t")
        record = dict(zip(header, fields))
        records.append(record)
refseq_summary = pl.from_records(records).rename({"taxid": "ncbi_taxonid"})
refseq_summary.collect_schema()
# Schema([('assembly_accession', String),
#         ('bioproject', String),
#         ('biosample', String),
#         ('wgs_master', String),
#         ('refseq_category', String),
#         ('ncbi_taxonid', String),
#         ('species_taxid', String),
#         ('organism_name', String),
#         ('infraspecific_name', String),
#         ('isolate', String),
#         ('version_status', String),
#         ('assembly_level', String),
#         ('release_type', String),
#         ('genome_rep', String),
#         ('seq_rel_date', String),
#         ('asm_name', String),
#         ('asm_submitter', String),
#         ('gbrs_paired_asm', String),
#         ('paired_asm_comp', String),
#         ('ftp_path', String),
#         ('excluded_from_refseq', String),
#         ('relation_to_type_material', String),
#         ('asm_not_live_date', String),
#         ('assembly_type', String),
#         ('group', String),
#         ('genome_size', String),
#         ('genome_size_ungapped', String),
#         ('gc_percent', String),
#         ('replicon_count', String),
#         ('scaffold_count', String),
#         ('contig_count', String),
#         ('annotation_provider', String),
#         ('annotation_name', String),
#         ('annotation_date', String),
#         ('total_gene_count', String),
#         ('protein_coding_gene_count', String),
#         ('non_coding_gene_count', String),
#         ('pubmed_id', String)])

refseq_summary.write_csv(
    os.path.join(rrna_dir, "refseq_assembly_summary.tsv"), separator="\t"
)
refseq_summary = pl.read_csv(
    os.path.join(rrna_dir, "refseq_assembly_summary.tsv"),
    infer_schema_length=100020,
    separator="\t",
    null_values=["na", "NA", "-"],
    ignore_errors=True,
    has_header=True,
)

# In [91]: refseq_summary.collect_schema()
# # Out[91]:
# Schema([('assembly_accession', String),
#         ('bioproject', String),
#         ('biosample', String),
#         ('wgs_master', String),
#         ('refseq_category', String),
#         ('ncbi_taxonid', Int64),
#         ('species_taxid', Int64),
#         ('organism_name', String),
#         ('infraspecific_name', String),
#         ('isolate', String),
#         ('version_status', String),
#         ('assembly_level', String),
#         ('release_type', String),
#         ('genome_rep', String),
#         ('seq_rel_date', String),
#         ('asm_name', String),
#         ('asm_submitter', String),
#         ('gbrs_paired_asm', String),
#         ('paired_asm_comp', String),
#         ('ftp_path', String),
#         ('excluded_from_refseq', String),
#         ('relation_to_type_material', String),
#         ('asm_not_live_date', String),
#         ('assembly_type', String),
#         ('group', String),
#         ('genome_size', Int64),
#         ('genome_size_ungapped', Int64),
#         ('gc_percent', Float64),
#         ('replicon_count', Int64),
#         ('scaffold_count', Int64),
#         ('contig_count', Int64),
#         ('annotation_provider', String),
#         ('annotation_name', String),
#         ('annotation_date', String),
#         ('total_gene_count', Int64),
#         ('protein_coding_gene_count', Int64),
#         ('non_coding_gene_count', Int64),
#         ('pubmed_id', String)])
refseq_summary.write_parquet(
    os.path.join(rrna_dir, "refseq_assembly_summary.parquet")
)

INFO     2025-12-28 21:17:12 - Downloading https://ftp.ncbi.nlm.nih.gov/genomes/refseq/assembly_summary_refseq.txt to                   ]8;id=867597;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py\various.py]8;;\:]8;id=118293;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py#215\215]8;;\
         /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/assembly_summary_refseq.txt.gz                                    

INFO     2025-12-28 21:17:20 - Successfully downloaded to                                                                               ]8;id=777524;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py\various.py]8;;\:]8;id=5085;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/utils/various.py#221\221]8;;\
         /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/assembly_summary_refseq.txt.gz                                    

Loading NCBI RefSeq assembly summary


In [ ]:
genbank_summary = pl.read_parquet(
    os.path.join(rrna_dir, "genbank_assembly_summary.parquet")
)
refseq_summary = pl.read_parquet(
    os.path.join(rrna_dir, "refseq_assembly_summary.parquet")
)
test = set(genbank_summary.columns).intersection(set(refseq_summary.columns))
print(test)
print(len(test))
# is refseqs in genbank?
refseq_in_genbank = refseq_summary.join(
    genbank_summary.select(["assembly_accession"]).unique(),
    on=["assembly_accession"],
    how="inner",
)
print(
    f"Total of {refseq_in_genbank.height} RefSeq assemblies found in GenBank assemblies"
)
ncbi_summary = pl.concat(
    [genbank_summary, refseq_summary], how="diagonal_relaxed"
).unique(subset=["assembly_accession"])
ncbi_summary.write_parquet(
    os.path.join(rrna_dir, "ncbi_assembly_summary.parquet")
)
ncbi_summary.write_csv(
    os.path.join(rrna_dir, "ncbi_assembly_summary.tsv"), separator="\t"
)
print(
    f"Total of {ncbi_summary.height} unique assemblies found in NCBI GenBank and RefSeq"
)

{'total_gene_count', 'version_status', 'annotation_date', 'non_coding_gene_count', 'biosample', 'pubmed_id', 'bioproject', 'species_taxid', 'assembly_type', 'scaffold_count', 'protein_coding_gene_count', 'assembly_accession', 'ncbi_taxonid', 'seq_rel_date', 'genome_size', 'asm_submitter', 'genome_size_ungapped', 'annotation_name', 'genome_rep', 'isolate', 'asm_name', 'wgs_master', 'replicon_count', 'refseq_category', 'infraspecific_name', 'organism_name', 'gbrs_paired_asm', 'group', 'assembly_level', 'excluded_from_refseq', 'paired_asm_comp', 'ftp_path', 'gc_percent', 'release_type', 'contig_count', 'asm_not_live_date', 'relation_to_type_material', 'annotation_provider'}
38
Total of 0 RefSeq assemblies found in GenBank assemblies
Total of 3718859 unique assemblies found in NCBI GenBank and RefSeq


Looks like they are not in genbank, so stacking them

In [ ]:
ncbi_summary = pl.read_parquet(
    os.path.join(rrna_dir, "ncbi_assembly_summary.parquet")
)

In [ ]:
# next, for every unique ncbi_taxonid, we select the one that has the most protein_coding_gene_count, then refseq_category, then tie breaking with non_coding_gene_count, tie breaking by latest assembly (by seq_rel_date).
mini_ncbi = ncbi_summary.sort(
    by=[
        pl.col("refseq_category").reverse(),
        pl.col("protein_coding_gene_count").cast(pl.Int64).reverse(),
        pl.col("non_coding_gene_count").cast(pl.Int64).reverse(),
        pl.col("seq_rel_date").reverse(),
        pl.col("genome_size").reverse(),
    ]
).unique(subset=["ncbi_taxonid"], keep="first")
print(
    f"Filtered GenBank summary to {mini_ncbi.height} unique taxid entries for SILVA sequences"
)

# # in_silva = mini_ncbi.filter(pl.col("ncbi_taxonid").is_in(silva_taxmap["ncbi_taxonid"])).unique()
# print(in_silva.height)
# print(temp_genbank.height)
# only 30503 out of ~240k?

Filtered GenBank summary to 244185 unique taxid entries for SILVA sequences


## rRNAs, tRNA, mtDNA, and plastid-DNA from NCBI BLAST DBs

This is a little hacky, but works?

rRNAs

In [ ]:
# first, getting the contents of https://ftp.ncbi.nlm.nih.gov/blast/db/ and filtering to files containing "16S", "18S", "28S", "ITS"
# LSU, SSU

terms = ["16S", "18S", "28S", "ITS", "LSU", "SSU", "ribosomal"]
files_2_download = []
FTP_HOST = "ftp.ncbi.nlm.nih.gov"
FTP_DIR = "/blast/db/"

# List to store the file and directory names
file_list = []

# Connect to the FTP host
with FTP(FTP_HOST) as ftp:
    #  Log in anonymously (default behavior for .login())
    ftp.login()
    print(f"Connected to {FTP_HOST}")
    ftp.cwd(FTP_DIR)
    logger.debug(f"Changed directory to {FTP_DIR}")

    # nlst() returns only names, not detailed information
    file_list = ftp.nlst()

    print("\n--- Directory Contents ---")
    for item in file_list:
        if any(term in item for term in terms):
            files_2_download.append(item)
            logger.debug(item)

            # Download file directly using FTP
            local_path = os.path.join(rrna_dir, item)
            print(f"Downloading {item}")
            with open(local_path, "wb") as local_file:
                ftp.retrbinary(f"RETR {item}", local_file.write)
            logger.debug(f"Successfully downloaded {item}")

print(f"Downloaded {len(files_2_download)} files matching terms: {terms}")

In [ ]:
# for each of these, will run blastdbcmd to extract sequences into fasta
for item in files_2_download:
    if not item.endswith(".tar.gz"):
        continue
    print(f"Processing {item}...")
    extract(archive_path=os.path.join(rrna_dir, item), extract_to=rrna_dir)
    # blastdbcmd -entry all -db LSU_prokaryote_rRNA  -out reference.fasta -outfmt "%T;%t;%s" # taxid;header;sequence
    run_command_comp(
        base_cmd="blastdbcmd",
        positional_args=[
            "-entry",
            "all",
            "-db",
            os.path.join(rrna_dir, item.replace(".tar.gz", "")),
            "-out",
            os.path.join(
                rrna_dir,
                item.replace(".tar.gz", "").replace(".tar", "") + ".tab",
            ),
            "-outfmt",
            str('"%T@%t@%K@%s"'),
        ],
        positional_args_location="end",
        params={},
        logger=logger,
        output_file=os.path.join(
            rrna_dir, item.replace(".tar.gz", "").replace(".tar", "") + ".tab"
        ),
    )

In [ ]:
# get all files to remove (everything but the .tab files)
all_files_in_rrna = glob.glob(os.path.join(rrna_dir, "*"))
all_tab_files = glob.glob(os.path.join(rrna_dir, "*.tab"))
all_fasta_files = glob.glob(os.path.join(rrna_dir, "*.fasta"))

# Files to keep (tab files, fasta files, and parquet/tsv files)
files_to_keep = set(
    all_tab_files
    + all_fasta_files
    + glob.glob(os.path.join(rrna_dir, "*.parquet"))
    + glob.glob(os.path.join(rrna_dir, "*.tsv"))
    + glob.glob(os.path.join(rrna_dir, "*.gz"))
)

# Get all files to remove (exclude directories and files to keep)
files_2_remove = [
    f for f in all_files_in_rrna if os.path.isfile(f) and f not in files_to_keep
]

print(f"Found {len(files_2_remove)} files to remove")
print(f"Keeping {len(files_to_keep)} files (.tab, .fasta, .parquet, .tsv, .gz)")

# Remove the files
for file_path in files_2_remove:
    try:
        os.remove(file_path)
        print(f"Removed: {os.path.basename(file_path)}")
    except Exception as e:
        print(f"Failed to remove {os.path.basename(file_path)}: {e}")

print("Cleanup complete")

Found 1 files to remove
Keeping 54 files (.tab, .fasta, .parquet, .tsv, .gz)
Removed: 18S_fungal_sequences.tar.gz.md5
Cleanup complete


In [ ]:
rrna_df = pl.scan_csv(
    os.path.join(rrna_dir, "*.tab"),
    separator="@",
    has_header=False,
    null_values=["N/A"],
    new_columns=["taxid", "header", "name", "sequence"],
    include_file_paths="type",
).collect()
rrna_df = rrna_df.drop("name")
rrna_df = rrna_df.with_columns(
    rRNA_type=pl.col("type").str.extract(r"([^/]+)\.tab$", 1)
)
rrna_df = rrna_df.drop("type")
print(f"Read {rrna_df.height} rRNA sequences from")

Read 159996 rRNA sequences from


In [ ]:
rrna_df["rRNA_type"].value_counts(sort=True)

rRNA_type,count
str,u32
"""ITS_eukaryote_sequences""",77582
"""16S_ribosomal_RNA""",27708
"""ITS_RefSeq_Fungi""",19541
"""28S_fungal_sequences""",11754
"""SSU_eukaryote_rRNA""",8784
"""LSU_eukaryote_rRNA""",6575
"""LSU_prokaryote_rRNA""",4047
"""18S_fungal_sequences""",4005


### rename fasta seqs to contam notation
for files to be used in filtering reads later on, the headers should bew parseable by other rolypoly functions. So renaming them here.
`taxid@Source@type@seq_md5_hash`
Using @ as separator as it's not common in fasta headers, and just going to assume the original header doesn't contain @, and that mappers who drop everything after the first white space will just ignore the really long headers
```fasta
>2888294@NCBI@ITS_RefSeq_Fungi@55c18f2cfc8490f5d5cb9b34162a44f5
atgtgatgaga...
```

In [ ]:
rrna_df = rrna_df.with_columns(
    t1=pl.concat_str(
        [pl.col("taxid"), pl.lit("NCBI"), pl.col("rRNA_type")], separator="@"
    )
)

rrna_df = rrna_df.with_columns(
    pl.col("sequence")
    .map_elements(lambda x: hashlib.md5(x.encode()).hexdigest())
    .alias("seq_hash")
).with_columns(
    new_header=pl.concat_str([pl.col("t1"), pl.col("seq_hash")], separator="@")
)
rrna_df = rrna_df.unique()
rrna_df = rrna_df.unique(subset=["sequence"])


rrna_df["new_header"].head(1).item()

'2888294@NCBI@ITS_RefSeq_Fungi@55c18f2cfc8490f5d5cb9b34162a44f5'

In [ ]:
rrna_df.height == rrna_df.with_columns(
    header_hash=pl.col("header").map_elements(
        lambda x: hashlib.md5(x.encode()).hexdigest()
    )
).select("header_hash").unique().height

False

Maybe some have more than 1 unique SSU/LSU?

In [ ]:
# rrna_df.with_columns(length=pl.col("sequence").str.len_chars()).select(pl.col("length").min()).item()

104

In [ ]:
rrna_fasta_path = os.path.join(rrna_dir, "ncbi_rRNA_all_sequences.fasta")

rrna_df = rrna_df.filter(
    ~pl.col("taxid").is_in(
        [77133, 155900, 115547, 256318, 100272, 136703, 171953, 86473, 56765]
    )
)
write_fasta_file(
    seqs=rrna_df["sequence"].to_list(),
    headers=rrna_df["new_header"].to_list(),
    output_file=rrna_fasta_path,
)
print(f"Wrote combined ncbi rRNA fasta to {rrna_fasta_path}")

Wrote combined ncbi rRNA fasta to /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/ncbi_rRNA_all_sequences.fasta


Next is the esame entropy and RNA virus masking as above.

In [ ]:
import rich_click as click

# first replace headers with md5 hash as the new_header breaks mappers. Will remap after this
write_fasta_file(
    headers=rrna_df["new_header"].to_list(),
    seqs=rrna_df["sequence"].to_list(),
    output_file=os.path.join(rrna_dir, "temp_ncbi_rRNA_all_sequences.fasta"),
)

# mask shared kmers with RNA viruses
masked_ncbi_fasta_path = os.path.join(
    rrna_dir, "temp_ncbi_rRNA_all_sequences_masked.fasta"
)

entrophy_masked_ncbi_fasta_path = os.path.join(
    rrna_dir, "temp_ncbi_rRNA_all_sequences_masked_entropy.fasta"
)

mask_args = {
    "threads": 15,
    "memory": "68gb",
    "output": str(masked_ncbi_fasta_path),
    "flatten": False,
    "aligner": "mmseqs2",
    "input": os.path.join(rrna_dir, "temp_ncbi_rRNA_all_sequences.fasta"),
}

# ensure we have the click Command object (the package import may have bound the module)
from rolypoly.commands.reads.mask_dna import mask_dna as _mask_cmd

# If it's a click.Command use a Context to invoke, otherwise call directly
if isinstance(_mask_cmd, click.core.Command):
    ctx = click.Context(_mask_cmd)
    ctx.invoke(_mask_cmd, **mask_args)
else:
    # fallback: call the function directly
    _mask_cmd(**mask_args)

# # Apply entropy masking
bbduk(
    in1=masked_ncbi_fasta_path,
    out=entrophy_masked_ncbi_fasta_path,
    entropy=0.4,
    entropyk=4,
    entropywindow=24,
    maskentropy=True,
    threads=15,
)

# read the masked fasta and remap to original headers
ncbi_rrna_masked_df = from_fastx_eager(entrophy_masked_ncbi_fasta_path).rename(
    {"header": "new_header", "sequence": "masked_sequence"}
)
ncbi_rrna_masked_df = ncbi_rrna_masked_df.join(
    rrna_df, on=["new_header"], how="inner"
)
ncbi_rrna_masked_df.write_parquet(
    os.path.join(rrna_dir, "ncbi_rrna_sequences_masked.parquet")
)
print("Finished writing NCBI rRNA sequences parquet file.")
write_fasta_file(
    headers=ncbi_rrna_masked_df["new_header"].to_list(),
    seqs=ncbi_rrna_masked_df["masked_sequence"].to_list(),
    output_file=os.path.join(
        rrna_dir, "ncbi_rRNA_all_sequences_masked_entropy.fasta"
    ),
)
print(
    f"finished writing masked NCBI rRNA fasta file: {os.path.join(rrna_dir, 'ncbi_rRNA_all_sequences_masked_entropy.fasta')}"
)

INFO     2025-12-28 22:47:10 - datadir used: /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data                             ]8;id=933960;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/commands/reads/mask_dna.py\mask_dna.py]8;;\:]8;id=343840;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/commands/reads/mask_dna.py#71\71]8;;\

Running command: mmseqs easy-linsearch /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/masking/combined_entropy_masked.fasta /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/temp_ncbi_rRNA_all_sequences.fasta /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/tmp_mask_dna/tmp_mapped.sam /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/tmp_mask_dna  --min-seq-id 0.7 --min-aln-len 80 --threads 15 --format-mode 1 --search-type 3 -v 3 --max-accept 1231 -a  
easy-linsearch /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/masking/combined_entropy_masked.fasta /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/temp_ncbi_rRNA_all_sequences.fasta /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/tmp_mask_dna/tmp_mapped.sam /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/tmp_mask_dna --min-seq-id 0.7 --m

File '/clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/tmp_mask_dna/tmp_mapped.sam' size 
is 191029 bytes (not empty). 

INFO     2025-12-28 22:47:35 - Finished running aligner mmseqs2                                                                        ]8;id=988782;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/commands/reads/mask_dna.py\mask_dna.py]8;;\:]8;id=121695;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/commands/reads/mask_dna.py#220\220]8;;\

INFO     2025-12-28 22:47:35 - beginning bbmask (masking + entropy) step                                                               ]8;id=867646;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/commands/reads/mask_dna.py\mask_dna.py]8;;\:]8;id=322806;file:///clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/src/rolypoly/commands/reads/mask_dna.py#221\221]8;;\

...

Result:                         145146 reads (100.00%)  160837643 bases (100.00%)

Time:                           0.816 seconds.

Reads Processed:        145k    177.86k reads/sec

Bases Processed:        160m    197.09m bases/sec

Finished writing NCBI rRNA sequences parquet file.
finished writing masked NCBI rRNA fasta file: /clusterfs/jgi/scratch/science/metagen/neri/code/rolypoly/data/contam/rrna/ncbi_rRNA_all_sequences_masked_entropy.fasta


In [ ]:
temp_df = ncbi_rrna_masked_df.join(
    mini_ncbi.select(["ncbi_taxonid", "organism_name"]).unique(),  # seq_length
    left_on=["taxid"],
    right_on=["ncbi_taxonid"],
    how="left",
)
temp_df["organism_name"].value_counts(sort=True)

organism_name,count
str,u32
null,105619
"""Globisporangium heterothallicu…",93
"""Babesia bovis""",65
"""Globisporangium sylvaticum""",61
"""Globisporangium irregulare""",56
…,…
"""Sphingobacterium deserti""",1
"""Pararcticibacter amylolyticus""",1
"""Leontodon rigens""",1


some 105k nulls, these are the ones that will need to patch to link to cloest related organism that does have genome/transcriptome data on genbank.  
That is done in [rrna_genome_mapping_taxonomy.ipynb](rrna_genome_mapping_taxonomy.ipynb)

## Now tRNAs

In [ ]:
file_url = (
    "https://ftp.ebi.ac.uk/pub/databases/Rfam/CURRENT/fasta_files/RF00005.fa.gz"
)
trna_seqs = os.path.join(trna_dir, "tRNA_sequences.fasta")
gz_filename = "RF00005.fa.gz"
fetch_and_extract(
    url=file_url,
    fetched_to=os.path.join(trna_dir, gz_filename),
    extract_to=trna_dir,
    expected_file=trna_seqs,
)
print(f"Downloaded tRNA sequences to {trna_seqs}")

In [ ]:
# remove duplicates
deduplicated_fasta = os.path.join(trna_dir, "tRNA_sequences_deduplicated.fasta")
remove_duplicates(
    input_file=trna_seqs,
    output_file=deduplicated_fasta,
    return_stats=True,
    by="seq",
)

In [ ]:
from rolypoly.utils.bio.polars_fastx import compute_aggregate_stats, fasta_stats

info_table = fasta_stats(deduplicated_fasta)
# print(info_table)

some_stats = compute_aggregate_stats(
    info_table, fields=["length", "gc_content", "n_content"]
)
some_stats

I don't trust this (min len 30, min gc 8% ), so will do some light filteration.

In [ ]:
info_table = info_table.filter(
    pl.col("length").is_between(60, 250), pl.col("gc_content") >= 0.01
)
some_stats_filtered = compute_aggregate_stats(
    info_table, fields=["length", "gc_content", "n_content"]
)
some_stats_filtered
# info_table.write_parquet(os.path.join(trna_dir, "tRNA_sequences_deduplicated_stats.parquet"))

looks a little better, will just accept that very low or very high GC content tRNAs exist.

In [ ]:
write_fasta_file(
    seqs=info_table["sequence"].to_list(),
    headers=info_table["header"].to_list(),
    output_file=os.path.join(
        trna_dir, "tRNA_sequences_deduplicated_filtered.fasta"
    ),
)
print(
    f"Wrote filtered tRNA sequences to {os.path.join(trna_dir, 'tRNA_sequences_deduplicated_filtered.fasta')}"
)

## plastid and mitochondria genomes

In [ ]:
from rolypoly.commands.misc.build_data import (
    prepare_plastid_data,
    prepare_mito_data,
)

prepare_plastid_data(data_dir, logger)
prepare_mito_data(data_dir, logger)